# Korteweg-deVries Equation

$$
u_t + u u_x + \mu u_{xxx} = 0 \qquad (0 < x < 1)
$$

See [Interaction of "Solitons" in a Collisionless Plasma and the Recurrence of Initial States
](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.15.240) (N. J. Zabusky and M. D. Kruskal
Phys. Rev. Lett., Vol. 15, 240, 1965).

It does not seem that the solution returns to the initial state exactly, though it gets close. I don't know why. More accurate schemes might be needed.

## Importing modules 

In [26]:
import time

import numpy as np
import equations as eq

import h5py

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.widgets import Slider

In [27]:
%matplotlib widget

## Setting Parameters 

In [28]:
NW = 50
kdv01 = eq.KdV(NW)
args = kdv01.getParamDefault()

J = kdv01.J
x = kdv01.sc.get_x()

## Setting initial data 

In [29]:
w0p = np.cos(2.0*np.pi*x).reshape((1,J))
w0c = kdv01.sc.transform_wp2wc(w0p)

## Preparation

In [30]:
trange = np.linspace(0., 6.0, 601)
max_step = 1.0e-1
fname = 'kdv.hdf5'

## Computation

In [ ]:
with h5py.File(fname, 'w') as fh:
    kdv01.mkInitDataSet(w0c, fh)
    kdv01.evolve(fh, trange, tuple(args), 
                max_step=max_step, method='Radau', pb_type="notebook")

## Visualization

In [32]:
with h5py.File(fname, 'r') as fh:
    wp = fh['wp'][()]
    powers = fh['powerspec'][()]
    atime = fh['trange'][()]

In [33]:
freq_max = NW
freq = np.arange(NW)

In [ ]:
gsWS = GridSpec(1,2)
gsTimeSlider = GridSpec(2,1)
gsWS.update(bottom=0.25, top=0.8, right=0.8)
gsTimeSlider.update(bottom=0.05, top=0.2)

fig = plt.figure(figsize=(6,6));

ax_ph = fig.add_subplot(gsWS[0,0])
ax_ph.set_ylim((-1.2, 3.0))
ax_ph.set_xlim((-0.1, 1.1))
ax_ph.grid(True)
ax_ph.set_xlabel('x')
ax_ph.set_ylabel('u')

ax_spec = fig.add_subplot(gsWS[0,1])
ax_spec.yaxis.set_ticks_position('right')
ax_spec.yaxis.set_label_position('right')
ax_spec.set_xlim((0., NW))
ax_spec.set_ylim(-0.05, 0.3)
ax_spec.set_xlabel('mode')
ax_spec.set_ylabel('power')
ax_spec.grid(True)


lines = ax_ph.plot(x, [np.nan]*len(x));

pow_specs = ax_spec.plot(
    freq[:freq_max],
    [np.nan]*freq_max,
    color=(1.0, 0.0, 0.0, 0.4)
    )

timeframe = fig.add_subplot(gsTimeSlider[0,0])
timeframe.set_axis_off()
timer = timeframe.text(0.5,0.5,
                       '', 
                       va='center', ha="center"
                      )
timeformat = "time = {0:05.3f}"

slider = Slider(
    ax = fig.add_subplot(gsTimeSlider[1,0]),
    valmin=0,
    valmax= wp.shape[0] - 1,
    label="time index",
    valstep=1,
    valinit=0,
    orientation="horizontal"
)

def _update(idx):
    for l in lines:
        l.set_ydata(wp[idx, 0, :]);
    for p in pow_specs:
        p.set_ydata(powers[idx, 0, :freq_max])
    timer.set_text(timeformat.format(atime[idx]))

slider.on_changed(_update)

